In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)
filename_grokKey = "..\\GroqAPIKey.txt"
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)
filename_langsmithKey = "..\\LangSmithAPIKey.txt"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = get_file_contents(filename_langsmithKey)
os.environ['LANGCHAIN_PROJECT'] = "first_project"


### LangSmith for monitoring the LLM applications

In [4]:
from langchain_groq import ChatGroq


In [5]:
#llm= ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
llm = ChatGroq(model="llama-3.1-8b-instant")

In [6]:
llm.invoke("What is langchain?")

AIMessage(content='Langchain is an open-source platform for building and deploying AI models, particularly those that utilize natural language processing (NLP) and large language models (LLMs). It was founded by David Tian, a well-known Canadian entrepreneur, and AI researcher.\n\nThe main goal of Langchain is to create a more accessible and user-friendly interface for developers, researchers, and users to build and interact with AI models. It aims to simplify the process of integrating and deploying LLMs in various applications, such as chatbots, virtual assistants, and content generation tools.\n\nSome key features of Langchain include:\n\n1. **LLM Integration**: Langchain provides a simple and efficient way to integrate LLMs from popular frameworks like Hugging Face, Transformers, and others.\n2. **Context-Aware**: Langchain allows developers to build context-aware AI models that can understand and respond to complex, multi-turn conversations.\n3. **Conversation Management**: It off

In [7]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq


In [8]:
#llm= ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
llm = ChatGroq(model="llama-3.1-8b-instant")

In [9]:
search = DuckDuckGoSearchResults()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [10]:
prompt = ChatPromptTemplate(
    [
        (
            "system","You are a helpful assistant. For answering the user query, look for information using DuckDuckGo Search and Wikipedia and then give the final answer."
        ),
        ("placeholder","{history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}"), # must add when using tool calling agent
    ]
)

In [11]:
tools = [search,wikipedia]


In [12]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [13]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True, input_keys=["input"])

In [14]:
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] =  ChatMessageHistory()
    return store[session_id]

In [15]:
agent_with_history = RunnableWithMessageHistory(
    agent_executor, 
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [16]:
print(agent_with_history.input_messages_key)

input


In [17]:
print(agent_with_history.history_messages_key)

history


In [18]:
agent_with_history.invoke(
    {"input":"When was Nelson Mandela born?"},
    config={"configurable":{"session_id":"sess1"}},
    )



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela birthdate'}`


Page: Pablo Sarino
Summary: Pablo Gomez Sarino (June 29, 1911 - December 23, 1987) was a Filipino politician who served as the municipal mayor of Bacoor, Cavite. He previously served as a municipal councilor and later became elected as the mayor in 1959. His terms of office in total made him the longest serving mayor of Bacoor to date.

Page: Dominic Waghorn
Summary: Dominic David Waghorn (born 1968, Lambeth), is a British journalist who is the Diplomatic Editor of Sky News and presenter of the channel's weekly international affairs analysis programme World View.  He was before that US Correspondent of Sky News, the 24-hour television news service operated by Sky Television, part of British Sky Broadcasting. He is based at Sky News' Washington Bureau. He was formerly Sky News' Asia Correspondent, based in Beijing and Middle East Correspondent, based in Jerusalem. He became Sky 

{'input': 'When was Nelson Mandela born?',
 'history': [],
 'output': 'Nelson Mandela was born on July 18, 1918.'}

In [19]:
agent_with_history.invoke(
    {"input":"When was Nelson Mandela born?"},
    config={"configurable":{"session_id":"sess1"}},
    )



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela birthdate'}`


Page: Pablo Sarino
Summary: Pablo Gomez Sarino (June 29, 1911 - December 23, 1987) was a Filipino politician who served as the municipal mayor of Bacoor, Cavite. He previously served as a municipal councilor and later became elected as the mayor in 1959. His terms of office in total made him the longest serving mayor of Bacoor to date.

Page: Dominic Waghorn
Summary: Dominic David Waghorn (born 1968, Lambeth), is a British journalist who is the Diplomatic Editor of Sky News and presenter of the channel's weekly international affairs analysis programme World View.  He was before that US Correspondent of Sky News, the 24-hour television news service operated by Sky Television, part of British Sky Broadcasting. He is based at Sky News' Washington Bureau. He was formerly Sky News' Asia Correspondent, based in Beijing and Middle East Correspondent, based in Jerusalem. He became Sky 

{'input': 'When was Nelson Mandela born?',
 'history': [HumanMessage(content='When was Nelson Mandela born?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nelson Mandela was born on July 18, 1918.', additional_kwargs={}, response_metadata={})],
 'output': 'Nelson Mandela was born on July 18, 1918.'}

In [20]:
result = agent_with_history.invoke(
    {"input":"What are some of his famous quotes?"},
    config={"configurable":{"session_id":"sess1"}},
    )



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela famous quotes'}`


Page: Ndaba Mandela
Summary: Ndaba Thembekile Zweliyajika Mandela (born 23 December 1982) is an author, mentor, spokesperson, entrepreneur, political consultant, and the grandson of Nelson Mandela. He was born in Soweto, South Africa.
Mandela is currently the co-founder and chairman of the Mandela Institute for Humanity.
He is active in keeping the legacy of his grandfather alive, and introducing Nelson Mandela to a younger generation. He is the co-founder and chairman of Africa Rising Foundation.  He is also the founder of the Mandela Project. He was part of the team for the Mandela Centenary in 2018.
From 2014 - 2021 Ndaba was Vice President of the Pan-African Youth Council which is now called the Pan African Youth Parliament which works closely with the African Union.
He is the longest serving global ambassador for UNAIDS, a specialized agency of the United Nations that 

In [21]:
print(result['output'])

Here are some of Nelson Mandela's famous quotes:

1. "The greatest glory in living lies not in never falling, but in rising every time we fall."
2. "For to be free is not merely to cast off one's chains, but to live in a way that respects and enhances the freedom of others."
3. "I have walked that long road to freedom. I have tried not to falter; I have made missteps along the way. But I have discovered the secret that after climbing a great hill, one only finds that there are many more hills to climb."
4. "Education is the most powerful weapon which you can use to change the world."
5. "The true character of a person is not seen in triumph, but in adversity."
6. "A nation should not be judged by how it treats its superior, but by how it treats its inferior."
7. "If you want to make peace with your enemy, you have to work with your enemy. Then he becomes your partner."
8. "The greatest glory in living lies not in never falling, but in rising every time we fall."
9. "I am not a saint, u